## Introduction
#### In many applications, data may be spread across number of files or databases, or be arranged in a for that is not easy to analyze.
#### Knowing how to combine, join and rearrange data is an important skill in the Data Analyst toolkit.

## Hierarchical Indexing
#### It is an important feature in pandas that enables you to have multiple index levels on an axis.
#### It provides a way to work with higher dimensional data in lower dimensional form.
#### When looking at a Series or DataFrame with multi-index, you will see "gaps" in the higher index, which means "same as the one above".

In [1]:
import pandas as pd
import numpy as np

data = pd.Series(np.random.randn(9),
                index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                      [1, 2, 3, 1, 3, 1, 2, 2, 3]])

data

a  1    1.407820
   2   -1.461955
   3    1.228871
b  1    0.154511
   3   -0.003190
c  1   -0.027980
   2   -0.095242
d  2   -0.562603
   3   -0.681595
dtype: float64

In [2]:
data.index

MultiIndex(levels=[['a', 'b', 'c', 'd'], [1, 2, 3]],
           labels=[[0, 0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 2, 0, 1, 1, 2]])

#### With hierarchical indexed objects, you can perform 'partial indexing', which enables us to concisely select subsets of data.
#### Selection is also possible for "inner" level of indexes.

In [3]:
data['b']

1    0.154511
3   -0.003190
dtype: float64

In [4]:
data['b':'c']

b  1    0.154511
   3   -0.003190
c  1   -0.027980
   2   -0.095242
dtype: float64

In [5]:
data.loc[['b', 'd']]

b  1    0.154511
   3   -0.003190
d  2   -0.562603
   3   -0.681595
dtype: float64

In [6]:
data.loc[:,2]

a   -1.461955
c   -0.095242
d   -0.562603
dtype: float64

#### Hierarchical indexing has important role in reshaping data and group=based operations. eg - forming pivot table.
#### You could rearrange data into a DataFrame using its 'unstack' method.
#### The inverse operation of stack is 'stack'.

In [7]:
data.unstack()

,1,2,3
a,1.407820,-1.461955,1.228871
b,0.154511,NaN,-0.003190
c,-0.027980,-0.095242,NaN
d,NaN,-0.562603,-0.681595


In [8]:
data.unstack().stack()

a  1    1.407820
   2   -1.461955
   3    1.228871
b  1    0.154511
   3   -0.003190
c  1   -0.027980
   2   -0.095242
d  2   -0.562603
   3   -0.681595
dtype: float64

#### In a DataFrame, either axis can have hierarchical index.
#### The hierarchical indexes can have names and they will be shown in the console output.
#### NOTE - Be careful not to mix-up index names with row labels.
#### With partial column indexing, we can select groups of columns.
#### A 'MultiIndex' can be created by itself and then reused.

In [9]:
frame = pd.DataFrame(np.arange(12).reshape((4,3)),
                    index=[['a', 'a', 'b', 'b'], [1,2,1,2]],
                    columns = [['Ohio', 'Ohio', 'Colorado'],
                              ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [10]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [11]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [12]:
pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']],
                      names = ['state', 'color'])

MultiIndex(levels=[['Colorado', 'Ohio'], ['Green', 'Red']],
           labels=[[1, 1, 0], [0, 1, 0]],
           names=['state', 'color'])

### Reordering and Sorting Levels
#### Sometimes, we may need to rearrange the order of the levels on an axis or sort the data by values in one specific level.
#### We can use 'swaplevel', which takes 2 level numbers or name and returns a new object with the levels interchanged. The data is unaltered.
#### 'sort_index' sorts the data using the values in a single level.
#### We often use sort_index when swapping levels so that result is lexicographically sorted.
#### NOTE - Data selection performance is better on hierarchically indexed objects if index is lexicographically sorted, starting from outermost level.

In [13]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [14]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [15]:
frame.swaplevel(0,1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

### Summary statistics by Level
#### Many Descriptive and Summary statistics on DataFrame and Series have 'level' option, where you can specify the level you want to aggregate by on an axis.
#### we can aggregate on either rows or columns. Under the hood, it utilizes pandas' 'groupby machinery'.

In [16]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [17]:
frame.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### Indexing with a DataFrame's columns
#### We may want to use 1 or more columns as Row Index in DataFrame or alternatively, move row index into columns.
#### The 'set_index' method can create a new DataFrame using 1 or more of its columns as index.
#### By default, columns are removed from the DataFrame, but we can also leave them in.
#### 'reset_index' is the opposite of set_index. It moves hierarchical index levels into columns.

In [18]:
frame = pd.DataFrame({'a': range(7), 'b':range(7,0,-1),
                     'c':['one', 'one', 'one', 'two', 'two',
                          'two', 'two'],
                      'd': [0,1,2,0,1,2,3]})
frame                 

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [19]:
frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [20]:
frame.set_index(['c','d'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [21]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## Combining and Merging Datasets
#### Data in pandas can be combined in many ways:
####     1. 'pandas.merge' connects rows in DataFrames based on 1 or more keys. This is similar to 'join' operations in SQL.
####     2. 'pandas.concat' concatenates or 'stacks' together objects along an axis.
####     3. 'combine_first' method enables splicing together overlapping data to fill in missing values in one object from another. 

### Database-Style DataFrame Joins
#### Merge or join combines datasets by linking rows using 1 or more keys. They are central to relational databasees.
#### The 'merge' function in pandas is the main entry point for using these operations on your dataset.
#### We can have a many-to-one join, where 1 object has multiple rows with same index labels whereas the other object has only 1 row for each label. We get a combination of the rows from both objects.

In [24]:
df1 = pd.DataFrame({'key': ['b','b','a','c','a','a','b'],
                   'data1':range(7)})
df2 = pd.DataFrame({'key':['a','b','d'],
                   'data2': range(3)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [25]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [26]:
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


#### It is not necessary to specify which column to join on. 'merge' uses overlapping column names as the keys. But it is a good practice to specify explicitly.
#### If column names are different in the 2 objects, you need to specify them in the merge explicitly.
#### By default, merge does an 'inner join' so keys in the result are intersection or common set found in both objects.There are other possible join options- 'left', 'right' and 'outer'.
#### 'Outer join' does a union of the keys, combining the effect of left and right joins.    

In [28]:
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [29]:
df3 = pd.DataFrame({'lkey': ['b','b','a','c','a','a','b'],
                   'data1':range(7)})
df4 = pd.DataFrame({'rkey':['a','b','d'],
                   'data2': range(3)})

pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [30]:
pd.merge(df1, df2, how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


#### Many-to-many merges have well-defined although not vey intuitive behaviour.
#### They form Cartesian product of the rows. eg- 3 rows on left and 2 rows on right for the same label will lead to 6 rows in the result.

In [33]:
df1 = pd.DataFrame({'key': ['b','b','a','c','a', 'b'],
                   'data1':range(6)})
df2 = pd.DataFrame({'key':['a','b','a','b','d'],
                   'data2': range(5)})

df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [34]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [35]:
pd.merge(df1, df2, on='key', how='left')

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


In [36]:
pd.merge(df1, df2, how='inner')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


#### To merge on multiple keys, pass a list of columns.
#### To determine the key combinations in the result, think an array of tuples formed by the multiple keys being used as a single join key.
#### NOTE - When joining columns on columns, indexes on passed DataFrame objects are discarded.
#### When merging, we may get overlapping column names. We can address manually by renaming the axis labels.
#### 'merge' has a 'suffixes' option for specifying strings to append to overlapping names in left and right objects.

In [38]:
left = pd.DataFrame({'key1':['foo', 'foo', 'bar'],
                    'key2':['one', 'two', 'one'],
                    'lval':[1, 2, 3]})
right = pd.DataFrame({'key1':['foo', 'foo', 'bar', 'bar'],
                     'key2': ['one', 'one', 'one', 'two'],
                     'rval': [4, 5, 6, 7]})

pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [39]:
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [40]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


### Merging on Index
#### In some scenarios, the index will contain the key(s) along which the objects need to be merged.
#### You can pass 'left_index=True' or 'right_index=True' or both to indicate which index should be used to merge.
#### The default method for merge is to intersect join keys. But we can instead form 'union' with an 'outer_join'.

In [43]:
left1 = pd.DataFrame({'key':['a','b','a','a','b','c'],
                     'value':range(6)})
right1 = pd.DataFrame({'group_val':[3.5, 7]}, index=['a','b'])

left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [44]:
right1

,group_val
a,3.5
b,7.0


In [45]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [46]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


#### Things are more complicated with hierarchically indexed data, as joining on index is implicitly multi-key merge.
#### You can indicate multiple columns to merge as a list. And handle duplicate data with 'how=outer'.
#### Using the indexes of both sides is also possible.

In [48]:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
                     'key2': [2000, 2001, 2002, 2001, 2002],
                     'data': np.arange(5.)})
righth = pd.DataFrame(np.arange(12).reshape((6,2)),
                     index=[['Nevada', 'Nevada', 'Ohio', 'Ohio',
                            'Ohio', 'Ohio'],
                           [2001, 2000, 2000, 2000, 2001, 2002]],
                     columns=['event1', 'event2'])

lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [49]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [50]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [51]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], 
         right_index=True, how='outer')

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


In [52]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                    index=['a', 'c', 'e'],
                    columns=['Ohio', 'Nevada'])
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                     index=['b','c','d','e'],
                     columns=['Missouri','Alabama'])

left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [53]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [54]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


#### Dataframe has a 'join' method for merging by index. It can also be used to combine together many DataFrame objects that have similar indexes but non-overlapping columns.
#### Mostly for legacy reasons, the join method performs a left join on the keys, preserving the left frame's ro index.
#### It also supports joining the index of passed DataFrame on one of the columns of the calling DataFrame.
#### For index-on-index merges, you can pass a list of DataFrames to join instead of using the 'concat' function. 

In [56]:
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [57]:
left1.join(right1, on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [58]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                      index=['a', 'c', 'e', 'f'],
                      columns=['New York', 'Oregon'])
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [59]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [60]:
left2.join([right2, another], how='outer')

C:\Users\adity\Anaconda3\lib\site-packages\pandas\core\frame.py:6359: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  verify_integrity=True)


,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
b,NaN,NaN,7.0,8.0,NaN,NaN
c,3.0,4.0,9.0,10.0,9.0,10.0
d,NaN,NaN,11.0,12.0,NaN,NaN
e,5.0,6.0,13.0,14.0,11.0,12.0
f,NaN,NaN,NaN,NaN,16.0,17.0


### Concatenating along an Axis
#### Another data concatenation method is interchangeably known as concatenation, binding or stacking.
#### NumPy has 'concatenate' which performs this on NumPy arrays.

In [62]:
arr = np.arange(12).reshape((3,4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [63]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

#### In pandas' objects, we have labelled axes, which help us in generalizing array concatenation.
#### There are a few things to think about before concatenating pandas objects:
####     1. If the 2 objects are indexed differently on the other axis, should we combine the distinct element or use only the shared values along that axis (intersection)?
####     2. Do the concatenated chunks need to be identifiable in the resulting object?
####     3. Does the 'concatenating axis' have sata that needs to be preserved? In general, integer labels should best be discarded during concatenation.
#### The 'concat' method provides a consistent way to address the above issues.
#### For 'Series' with no overlap, concat just glues the values and indexes.
#### By default, it works along 'axis=0', but if we pass 'axis=1', teh result will be a DataFrame (axis=1 is for columns).

In [66]:
s1 = pd.Series([0,1], index=['a', 'b'])
s2 = pd.Series([2,3,4], index=['c', 'd', 'e'])
s3 = pd.Series([5,6], index=['f', 'g'])

In [67]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [68]:
pd.concat([s1, s2, s3], axis=1)

C:\Users\adity\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


#### If there is no overlap on the other axis (like above example), we see a 'sorted union' (or outer join).
#### You can get an intersect by passing "join='inner'".
#### If you want to specify the axes to be used in 'other axes' do it with 'join_axes'.

In [70]:
s4 = pd.concat([s1, s3])
s4

a    0
b    1
f    5
g    6
dtype: int64

In [72]:
pd.concat([s1, s4], axis=1)

C:\Users\adity\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [73]:
pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0,0
b,1,1


In [74]:
pd.concat([s1, s4], axis=1, join_axes=[['a', 'c', 'b', 'e']])

,0,1
a,0.0,0.0
c,NaN,NaN
b,1.0,1.0
e,NaN,NaN


#### Another issue is to identify concatenated pieces in the result. To create a hierarchical index in the concatenated axis, we can use the 'keys' argument.
#### When we use axis=1, the keys become column headers. This is true to both Series and DataFrame.
#### If we pass a dict of objects instead of a list, then dict's keys will be used for the 'keys' option.
#### There are other arguments in hierarchical index creation. eg - To name a created axis just use the 'names' argument.

In [76]:
result = pd.concat([s1, s2, s3], keys=['one', 'two', 'three'])
result

one    a    0
       b    1
two    c    2
       d    3
       e    4
three  f    5
       g    6
dtype: int64

In [77]:
result.unstack()

,a,b,c,d,e,f,g
one,0.0,1.0,NaN,NaN,NaN,NaN,NaN
two,NaN,NaN,2.0,3.0,4.0,NaN,NaN
three,NaN,NaN,NaN,NaN,NaN,5.0,6.0


In [78]:
pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'])

C:\Users\adity\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [80]:
df1 = pd.DataFrame(np.arange(6).reshape(3,2), index=['a', 'b', 'c'],
                  columns=['one', 'two'])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2,2), index=['a', 'c'],
                  columns=['three', 'four'])

df1

,one,two
a,0,1
b,2,3
c,4,5


In [81]:
df2

,three,four
a,5,6
c,7,8


In [82]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

C:\Users\adity\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [83]:
pd.concat({'level1':df1, 'level2': df2}, axis=1)

C:\Users\adity\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [84]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'],
         names=['upper', 'lower'])

C:\Users\adity\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  


upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

#### The last issue is that the row index does not contain any relevant data.
#### To avoid this, pass 'ignore_index=True'.

In [86]:
df1 = pd.DataFrame(np.random.randn(3,4), columns=['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(np.random.randn(2,3), columns=['b', 'd', 'a'])
df1

,a,b,c,d
0,-0.588973,1.051040,1.465350,2.065064
1,0.629663,0.600635,-0.489396,0.897867
2,-1.732375,-0.397218,-1.519736,-1.329098


In [87]:
df2

,b,d,a
0,0.279414,-1.949706,-0.988094
1,1.090094,1.267435,-0.928241


In [88]:
pd.concat([df1, df2], ignore_index=True)

C:\Users\adity\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


,a,b,c,d
0,-0.588973,1.051040,1.465350,2.065064
1,0.629663,0.600635,-0.489396,0.897867
2,-1.732375,-0.397218,-1.519736,-1.329098
3,-0.988094,0.279414,NaN,-1.949706
4,-0.928241,1.090094,NaN,1.267435


### Combining Data with Overlap
#### There is another data operation involving 2 datasets with indexes fully or partly overlapped.
#### NumPy has 'where' function which performs array-equivalent of an 'if-else'.
#### pandas has its equivalent 'combine_first'. In Series, this will perform the equivalent operation with the data alignment logic of pandas.
#### In DataFrames, it does the same column by column. So it is "patching" missing data from calling object with values from passed object.

In [90]:
a = pd.Series([np.nan, 2.5, 0.0, 3.5, 4.5, np.nan],
             index=['f','e','d','c','b','a'])
b = pd.Series([0., np.nan, 2., np.nan, np.nan, 5.],
             index=['a','b','c','d','e','f'])

a

f    NaN
e    2.5
d    0.0
c    3.5
b    4.5
a    NaN
dtype: float64

In [91]:
b

a    0.0
b    NaN
c    2.0
d    NaN
e    NaN
f    5.0
dtype: float64

In [93]:
np.where(pd.isnull(a), b, a)

array([0. , 2.5, 0. , 3.5, 4.5, 5. ])

In [94]:
b.combine_first(a)

a    0.0
b    4.5
c    2.0
d    0.0
e    2.5
f    5.0
dtype: float64

In [96]:
df1 = pd.DataFrame({'a':[1., np.nan, 5., np.nan],
                   'b':[np.nan, 2., np.nan, 6.],
                   'c':range(2,18,4)})
df2 = pd.DataFrame({'a':[5., 4., np.nan, 3., 7.],
                   'b':[np.nan, 3., 4., 6., 8.]})

df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [97]:
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [98]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


## Reshaping and Pivoting
#### There are a number of basic operations for rearranging tabular data. These are called 'reshape' or 'pivot' operations.

### Reshaping with Hierarchical Indexing
#### Hierarchical indexing allows to rearrange data consistently in a DataFrame. There are 2 primary actions:
####     1. stack - It 'rotates' or pivots from columns to rows.
####     2. unstack - Pivots rows into columns.
#### Calling 'stack' pivots columns into rows, producing a Series.
#### If you have a hierarchically indexed Series, we can rearrange the data back into a DataFrame with 'unstack'. 

In [101]:
data = pd.DataFrame(np.arange(6).reshape((2,3)),
                   index=pd.Index(['Ohio', 'Colorado'], name='state'),
                   columns = pd.Index(['one', 'two', 'three'],
                                     name = 'number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [103]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [104]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


#### By default the innermost index gets 'stacked' or 'unstacked'. You can unstack a different level by passing a level number or name.
#### Unstacking might introduce missing data if all values in the level aren't foundin each subgroups.

In [106]:
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [107]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [108]:
s1 = pd.Series([0,1,2,3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4,5,6], index=['c','d','e'])
data2 = pd.concat([s1, s2], keys=['one', 'two'])

data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [109]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


#### Stacking filters out missing data by default, so that the operation is easily invertible.
#### When you unstack, the level unstacked becomes lowest level in the result.
#### when calling stack or unstack, we can indicate the name of the axis to the stack.

In [111]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [113]:
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [114]:
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [115]:
df = pd.DataFrame({'left': result, 'right': result+5},
                 columns=pd.Index(['left', 'right'], name='side'))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [116]:
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [117]:
df.unstack('state').stack('side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

### Pivoting "Long" to "Wide" Format
#### A common way to store multiple time series in the so-called 'long' or 'stacked' format.
#### One way to store time index is via 'PeriodIndex', which combines year and quarter columns to create a kind of time interval type.
#### we use this interval to index our data.

In [124]:
data = pd.read_csv('examples/macrodata.csv')
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [125]:
data.columns

Index(['year', 'quarter', 'realgdp', 'realcons', 'realinv', 'realgovt',
       'realdpi', 'cpi', 'm1', 'tbilrate', 'unemp', 'pop', 'infl', 'realint'],
      dtype='object')

In [126]:
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter,
                        name='date')
columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')
data = data.reindex(columns=columns)
data.index = periods.to_timestamp('D', 'end')
ldata = data.stack().reset_index().rename(columns={0:'value'})

In [128]:
ldata[:5]

,date,item,value
0,1959-03-31,realgdp,2710.349
1,1959-03-31,infl,0.000
2,1959-03-31,unemp,5.800
3,1959-06-30,realgdp,2778.801
4,1959-06-30,infl,2.340


#### The above way of displaying data is called 'long format' for multiple time series or other observational data with 2or more keys.
#### Each row in the table is a single observation.
#### Data is frequently stored in this format in RDBMS as a fixed schema as it allows number of distinct values in the 'item' column to change as more data is added to the table.
#### In the previous example, date and item would be the primary keys, offering both relational integrity and easier joins.
#### But in some cases, this kind of data would be more difficult to work with, as you might prefer a DataFrame containing 1 column per distinct item indexed by timestamps.
#### This is exactly what 'pivot' method provides.

In [134]:
pivoted = ldata.pivot('date', 'item', 'value')
pivoted[:10]

item,infl,realgdp,unemp
date,,,
1959-03-31,0.00,2710.349,5.8
1959-06-30,2.34,2778.801,5.1
1959-09-30,2.74,2775.488,5.3
1959-12-31,0.27,2785.204,5.6
1960-03-31,2.31,2847.699,5.2
1960-06-30,0.14,2834.390,5.2
1960-09-30,2.70,2839.022,5.6
1960-12-31,1.21,2802.616,6.3
1961-03-31,-0.40,2819.264,6.8


#### The first 2 values passed in the pivot method are the columns to be used as 'row' and 'column' index.
#### If you had 2 'value' columns that you needed to reshape simultaneously. In pivot, by omitting the last argument, you get a DataFrame with hierarchical columns.
#### Using pivot is just like creating a hierarchical index using the 'set_index' followed by a call to 'unstack'.

In [136]:
ldata['value2'] = np.random.randn(len(ldata))
ldata[:5]

,date,item,value,value2
0,1959-03-31,realgdp,2710.349,-0.129071
1,1959-03-31,infl,0.000,-0.997675
2,1959-03-31,unemp,5.800,-0.388757
3,1959-06-30,realgdp,2778.801,-0.892829
4,1959-06-30,infl,2.340,-1.909896


In [137]:
pivoted = ldata.pivot('date', 'item')
pivoted[:5]

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-03-31  0.00  2710.349   5.8 -0.997675 -0.129071 -0.388757
1959-06-30  2.34  2778.801   5.1 -1.909896 -0.892829  0.348295
1959-09-30  2.74  2775.488   5.3 -0.327600 -0.433838  0.236335
1959-12-31  0.27  2785.204   5.6 -0.903713  0.328514 -1.249765
1960-03-31  2.31  2847.699   5.2 -1.634700 -0.421725 -0.299042

In [138]:
pivoted['value'][:5]

item,infl,realgdp,unemp
date,,,
1959-03-31,0.00,2710.349,5.8
1959-06-30,2.34,2778.801,5.1
1959-09-30,2.74,2775.488,5.3
1959-12-31,0.27,2785.204,5.6
1960-03-31,2.31,2847.699,5.2


In [139]:
unstacked = ldata.set_index(['date', 'item']).unstack('item')
unstacked[:5]

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-03-31  0.00  2710.349   5.8 -0.997675 -0.129071 -0.388757
1959-06-30  2.34  2778.801   5.1 -1.909896 -0.892829  0.348295
1959-09-30  2.74  2775.488   5.3 -0.327600 -0.433838  0.236335
1959-12-31  0.27  2785.204   5.6 -0.903713  0.328514 -1.249765
1960-03-31  2.31  2847.699   5.2 -1.634700 -0.421725 -0.299042

### Pivoting "Wide" to "Long" Format
#### The inverse to 'pivot' is 'pandas.melt'. It merges multiple columns into one, producing a DataFrame that is longer than the input.
#### When using pandas.melt, we indicate which columns, if any, are group indicators. In this example, we will use 'key' as the group indicator.

In [145]:
df =pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                 'A': [1,2,3],
                 'B': [4,5,6],
                 'C': [7,8,9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [146]:
melted = pd.melt(df, ['key'])
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


#### We can use pivot to get the original layout back. But because it creates an index from the columns used as row labels, we may have to use 'reset_index' to move it back into a column.
#### You can be explicit in which columns can be indicators and which can be values using the 'id_vars' and 'value_vars' parameters respectively.
#### We can use pandas.melt without group identifiers too.

In [148]:
reshaped = melted.pivot('key', 'variable', 'value')
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [149]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [150]:
pd.melt(df, id_vars=['key'], value_vars=['A','B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [151]:
pd.melt(df, value_vars=['A', 'B', 'C'])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [152]:
pd.melt(df, value_vars=['key', 'A', 'B'])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6
